# 1. Importar Librerías

In [196]:
import pandas as pd
import numpy as np
import sqlite3 as sql
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from a_funciones import ejecutar_sql

# 2. Importar Base de Datos

In [197]:
conn = sql.connect('data/db_movies')
cur = conn.cursor()

In [198]:
cur.execute('select name from sqlite_master where type = "table"')
cur.fetchall()

[('ratings',), ('movies',), ('movies2',), ('ratings2',), ('movies_rating',)]

In [199]:
df_movies = pd.read_sql('SELECT * FROM movies', conn)
df_ratings = pd.read_sql('SELECT * FROM ratings', conn)

# 3. Exploración previa movies

In [200]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [201]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


# 4. Exploración previa ratings

In [202]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [203]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


### 4.1. Distribución de ratings

In [204]:
df_ratings[['rating']].describe()

,rating
count,100836.000000
mean,3.501557
std,1.042529
min,0.500000
25%,3.000000
50%,3.500000
75%,4.000000
max,5.000000


In [205]:
query = '''
SELECT rating, count(userId) as Frecuencia
FROM ratings
GROUP BY rating
ORDER BY rating ASC;
'''
pd.read_sql(query, conn)

,rating,Frecuencia
0,0.5,1370
1,1.0,2811
2,1.5,1791
3,2.0,7551
4,2.5,5550
5,3.0,20047
6,3.5,13136
7,4.0,26818
8,4.5,8551
9,5.0,13211


In [206]:
fig = make_subplots(rows = 1, cols = 2)

fig.add_trace(
    go.Histogram(x = df_ratings.rating, name = 'Histograma ratings'),
    row = 1, col = 1
)

fig.add_trace(
    go.Box(y = df_ratings.rating, name = 'Boxplot ratings'),
    row = 1, col = 2
)

fig.update_layout(showlegend = False, title = 'Distribución de ratings', template = 'seaborn')
fig.show()

### 4.2. Películas vistas por usuarios

In [207]:
query = '''
SELECT count(*) as vistas
FROM ratings
GROUP BY userId;
'''
df_vistas_user = pd.read_sql(query, conn)

In [208]:
df_vistas_user.describe()

,vistas
count,610.000000
mean,165.304918
std,269.480584
min,20.000000
25%,35.000000
50%,70.500000
75%,168.000000
max,2698.000000


In [209]:
fig = make_subplots(rows = 1, cols = 2)

fig.add_trace(
    go.Histogram(x = df_vistas_user.vistas, name = 'Histograma vistas'),
    row = 1, col = 1
)

fig.add_trace(
    go.Box(y = df_vistas_user.vistas, name = 'Boxplot vistas'),
    row = 1, col = 2
)

fig.update_layout(showlegend = False, title = 'Distribución de total de películas vistas por usuario', template = 'seaborn')
fig.show()

# 5. Preprocesamiento

In [231]:
ejecutar_sql('d_Limpieza y Preprocesamiento.sql', cur)

In [232]:
movies_rating = pd.read_sql('SELECT * FROM movies_rating', conn)
movies_rating['date'] = pd.to_datetime(movies_rating['date'])
movies_rating['movieId'] = movies_rating['movieId'].astype(object)
movies_rating['userId'] = movies_rating['userId'].astype(object)
movies_rating['year'] = movies_rating['year'].astype(object)

In [233]:
movies_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 26 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   movieId      100836 non-null  object        
 1   year         100836 non-null  object        
 2   title        100836 non-null  object        
 3   Action       100836 non-null  int64         
 4   Adventure    100836 non-null  int64         
 5   Animation    100836 non-null  int64         
 6   Children     100836 non-null  int64         
 7   Comedy       100836 non-null  int64         
 8   Crime        100836 non-null  int64         
 9   Documentary  100836 non-null  int64         
 10  Drama        100836 non-null  int64         
 11  Fantasy      100836 non-null  int64         
 12  Film_Noir    100836 non-null  int64         
 13  Horror       100836 non-null  int64         
 14  IMAX         100836 non-null  int64         
 15  Musical      100836 non-null  int6

# 6. Análisis Exploratorio

### 6.1. Distribución vistas por género

In [238]:
query = 'SELECT '

for col in movies_rating.select_dtypes(int):
    query += f'sum({col}) as {col}, '

query = query[:-2] + ' FROM movies_rating;'
df_genres = pd.read_sql(query, conn).T.reset_index().rename(
            columns = {'index':'Género', 0:'Frecuencia'}).sort_values(
            by = 'Frecuencia', ascending = False)
            
df_genres.to_sql('genres', conn, if_exists='replace', index=False) # Crear una tabla con los géneros
df_genres

,Género,Frecuencia
7,Drama,41928
4,Comedy,39053
0,Action,30635
16,Thriller,26452
1,Adventure,24161
14,Romance,18124
15,Sci_Fi,17243
5,Crime,16681
8,Fantasy,11834
3,Children,9208


In [235]:
fig = px.bar(df_genres, x = 'Género', y = 'Frecuencia',
             title = 'Total de vistas por género',
             text = 'Frecuencia', template = 'seaborn')
fig.show()